<a href="https://colab.research.google.com/github/hargagan/EDA-NYC-Taxi-Data-Analysis/blob/main/pyspark/C3_M2_Practice_exercise_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('WeatherApp').getOrCreate()

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
df_weather = spark.read.csv('/content/drive/MyDrive/Assignments/EDA/Weather Dataset - CSV(in).csv', header=True, inferSchema=True)

In [9]:
df_weather.printSchema()

root
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: string (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: string (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: string (nullable = true)
 |-- WindSpeed3pm: integer (nullable = true)
 |-- Humidity9am: integer (nullable = true)
 |-- Humidity3pm: integer (nullable = true)
 |-- Pressure9am: double (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- Cloud9am: integer (nullable = true)
 |-- Cloud3pm: integer (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)



In [10]:
df_weather.show(5)

+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|
+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|    8.0|   24.3|     0.0|        3.4|     6.3|         NW|           30|        SW|        NW|           6|          20|         68|         29|     1019.7|     1015.0|       7|       7|   14.4|   23.6|       No|    3.6|         Yes|
|   14.0|   26.9|     3.6|        4.4|     9.7|        ENE| 

###**Task 1:** Write a function that returns the number of days when it rained the next day. Hint: Count the number of rows where RainTomorrow is ‘Yes’.


In [12]:
def count_rainy_day_tomorrow(df):
  return df.filter("RainTomorrow = 'Yes'").count()

count_rainy_day_tomorrow(df_weather)

66

###**Task 2:** Write a function that returns the average sunshine duration on days with no rainfall. Hint: Filter rows where Rainfall is 0 and then calculate the average of Sunshine.

In [25]:
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType

def avg_sunshine_duration_on_days_with_no_rainfall(df):
    # Replace 'NA' string with null and cast to DoubleType
    df_cleaned = df.withColumn("Sunshine", when(col("Sunshine") == "NA", None).otherwise(col("Sunshine").cast(DoubleType())))
    # Calculate the average and round it to 2 decimal places
    avg_value = df_cleaned.filter("Rainfall == 0.0").agg({"Sunshine": "avg"}).collect()[0][0]
    return round(avg_value, 2)

avg_sunshine_duration_on_days_with_no_rainfall(df_weather)

8.47

####**Task 3:** Write a function that returns the maximum temperature recorded at 3 PM. Hint: Use the Temp3pm column and return the max value.

In [27]:
df_weather.show(5)
def get_max_weather_3pm(df):
  df.createOrReplaceTempView("weather")
  return spark.sql("SELECT MAX(Temp3pm) FROM weather").collect()[0][0]

get_max_weather_3pm(df_weather)


+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|
+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|    8.0|   24.3|     0.0|        3.4|     6.3|         NW|           30|        SW|        NW|           6|          20|         68|         29|     1019.7|     1015.0|       7|       7|   14.4|   23.6|       No|    3.6|         Yes|
|   14.0|   26.9|     3.6|        4.4|     9.7|        ENE| 

34.5

###**Task 4:** Write a function that returns the average humidity at 3 PM on days it rained the next day. Hint: Filter rows where RainTomorrow is ‘Yes’ and then calculate the mean of Humidity3pm.

In [32]:
def avg_hum_at_3pm_for_day_rained_next_day(df):
  return df.filter("RainTomorrow = 'Yes'").agg({"Humidity3pm": "avg"}).collect()[0][0]

avg_hum_at_3pm_for_day_rained_next_day(df_weather)

57.68181818181818

#### **Task 5:** Write a function that returns the most common wind direction at 9 AM on cloudy days. Hint: Filter rows where Cloud9am is above a threshold (e.g. >5) and then find the mode of WindDir9am.

In [33]:
def common_wind_direction_9am_on_cloudy_day(df):
  return df.filter("Cloud9am>5").agg({"WindDir9am": "mode"}).collect()[0][0]

common_wind_direction_9am_on_cloudy_day(df_weather)

'SSE'